## Prepare model

### Imports & Definitions

In [59]:
import warnings

warnings.filterwarnings(
    "ignore",
    message="torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.",
)

warnings.filterwarnings(
    "ignore",
    message="torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.",
)

warnings.filterwarnings(
    "ignore",
    message="You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset",
)

In [60]:
from huggingface_hub import notebook_login
notebook_login()

In [61]:
import sys
sys.path.append("../../utils")
from definitions import *

In [62]:
CACHE_DIR = Path("../../../../cache/")
DATASET_DIR = Path("/home/jupyter/mnt/datasets/diplomas/russian_dataset/")

In [77]:
MODEL_EXPERIMENT_NAME = "learnt_8k_retest"

### Load model & tokenizer

In [81]:
model_name = LLAMA_2_7B
model = AutoModelForCausalLM.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name], 
    cache_dir=CACHE_DIR, 
    device_map='auto'
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

MODEL_MAX_LENGTH = 16384

tokenizer = AutoTokenizer.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name], 
    cache_dir=CACHE_DIR, 
    model_max_length=MODEL_MAX_LENGTH,
    padding_side="right",
    use_fast=True
)

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"

def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg

special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_tokens_dict,
    tokenizer=tokenizer,
    model=model,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.58s/it]
Using pad_token, but it is not set yet.


In [82]:
from peft import PeftModel

model_id = "nvdenisov2002/llama-longLoRA-v2"
peft_model = PeftModel.from_pretrained(model, model_id)

## Inference model on tasks

In [66]:
first_k = int(1e5)

### Inference model on asessors questions

In [83]:
from llm_helpers import get_some_model_result

test_df = pd.read_csv(ARTIFACTS_DIR_PATH.joinpath("diplomas_abstracts/mcs_raw_learnt_abstract_learnt8k.csv"))

new_rows = []
for _, row in tqdm(test_df[:first_k].iterrows(), total=len(test_df[:first_k]), desc="Rows..."):
    new_row = copy.deepcopy(row)
    new_row[MODEL_EXPERIMENT_NAME] = get_some_model_result(peft_model, tokenizer, row, device, diploma_prefix_len=8000)
    new_rows.append(new_row)
new_df = pd.DataFrame(new_rows)
new_df.head()

Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 4316, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 4170, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 5251, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.29 GiB. GPU 

In [ ]:
new_df.to_csv(ARTIFACTS_DIR_PATH.joinpath(f"diplomas_abstracts/baselines_with_{MODEL_EXPERIMENT_NAME}.csv"))

In [ ]:
test_df.__len__()

### Inference model on MERA

In [ ]:
from mera_helpers import construct_prompt
from path_helpers import get_dataset_path, get_metric_dir_path
from llm_helpers import calculate_token_interest_probs, get_answer

for name, dataset_meta in tqdm(HUGGINGFACE_NAME_TO_DATASET.items(), desc="Datasets..."):
    for subset, split in tqdm(zip(dataset_meta["subsets"], dataset_meta["splits"]), total=len(dataset_meta["splits"]), desc="Splits..."):
        path = get_dataset_path(subset, name, split)
        dataset = load_from_disk(path)
        probs_list = []
        a_list = []
        for row in tqdm(list(dataset)[:first_k], desc="Rows..."):
            q = construct_prompt(row)
            probs = calculate_token_interest_probs(q, tokenizer, peft_model)
            probs_list.append({
                "probs": probs,
                "meta": row["meta"],
            })
            a = get_answer(probs)
            a_list.append({
                "answer": a,
                "meta": row["meta"],
            })
        print(a_list)
        metric_dir_path = get_metric_dir_path(MODEL_EXPERIMENT_NAME, subset, name, split)
        print(metric_dir_path)
        metric_dir_path.mkdir(exist_ok=True, parents=True)
        with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
            json.dump(probs_list, f, ensure_ascii=False, indent=2)
        with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
            json.dump(a_list, f, ensure_ascii=False, indent=2)

### Inference & eval model on landmark passkey

In [ ]:
from landmark_run_test import test_passkey_full, load_pipes
from transformers import pipeline


models = [MODEL_EXPERIMENT_NAME]
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer)
pipes = {MODEL_EXPERIMENT_NAME: pipe}
test_passkey_full(pipes, models, n_values=[14000, 18000])

## Evaluate model results on tasks

### Evaluate asessors questions

In [ ]:
new_df = new_df.set_index("Unnamed: 0")
new_df

In [ ]:
with open(ARTIFACTS_DIR_PATH.joinpath("datasets/diplomas_asessors_questions/mcs_df_human_filled_processed.json"), "r") as f:
    dataset = json.load(f)

a_list = []
probs_list = []
for row in tqdm(dataset[:first_k], desc="Rows..."):
    x = copy.deepcopy(row)
    x['inputs']['context'] = new_df[MODEL_EXPERIMENT_NAME].loc[int(x['meta']['id'])] # df["diploma"].iloc[int(x['meta']['id'])]
    q = construct_prompt(x)
    probs = calculate_token_interest_probs(q, tokenizer, peft_model)
    probs_list.append({
        "probs": probs,
        "meta": row["meta"],
    })
    a = get_answer(probs)
    a_list.append({
        "answer": a,
        "meta": row["meta"],
    })
metric_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/{MODEL_EXPERIMENT_NAME}/{LLAMA_2_7B}/") # get_metric_dir_path(baseline_model, subset, name, split)
metric_dir_path.mkdir(exist_ok=True, parents=True)
with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
    json.dump(probs_list, f, ensure_ascii=False, indent=2)
with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
    json.dump(a_list, f, ensure_ascii=False, indent=2)

In [ ]:
rows = []
pred_by_model = dict()
for baseline_model in HUGGINFACE_BASELINE_MODELS:
    splits = ["abstract", "empty", "diploma"]
    if baseline_model == LLAMA_2_7B:
        splits.extend(["learnt", "learnt_8k", MODEL_EXPERIMENT_NAME])
    for split in splits:
        metric_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/{split}/{baseline_model}/") # get_metric_dir_path(baseline_model, subset, name, split)
        with open(metric_path.joinpath("answers.jsonl"), "r") as f:
            answers = json.load(f)
        pred = [x["answer"] for x in answers[:first_k]]
        pred_by_model[f"{split}_{baseline_model}"] = pred
        true = [x["outputs"] for x in dataset[:first_k]]
        rows.append({
            "model": baseline_model,
            "subset": "asessors_questions",
            "split": f"{split}_8000" if split == "diploma" else split,
            "accuracy_score": accuracy_score(true, pred),
        })
df = pd.DataFrame(rows)
df

In [ ]:
df.to_csv(METRICS_DIR_PATH.joinpath(f"asessors_with_{MODEL_EXPERIMENT_NAME}.csv"))

### Evaluate MERA

In [ ]:
rows = []
for baseline_model in HUGGINFACE_BASELINE_MODELS.union([VIKHR_7B, SAIGA_MISTRAL_7B_LORA]).union([MODEL_EXPERIMENT_NAME]):
    for name, dataset_meta in HUGGINGFACE_NAME_TO_DATASET.items():
        for subset, split in zip(dataset_meta["subsets"], dataset_meta["splits"]):
            dataset_path = get_dataset_path(subset, name, split)
            dataset = load_from_disk(dataset_path)
            metric_path = get_metric_dir_path(baseline_model, subset, name, split)
            with open(metric_path.joinpath("answers.jsonl"), "r") as f:
                answers = json.load(f)
            pred = [x["answer"] for x in answers[:first_k]]
            true = [x["outputs"] for x in list(dataset)[:first_k]]
            rows.append({
                "model": baseline_model,
                "subset": subset,
                "split": split,
                "accuracy_score": accuracy_score(true, pred),
            })
df = pd.DataFrame(rows)
df

In [ ]:
df.to_csv(METRICS_DIR_PATH.joinpath(f"ru_metrics_with_{MODEL_EXPERIMENT_NAME}.csv"))